In [14]:
# this chunk reads a csv from File section of Lakehouse into the notebook, creating the dataframe
# replace the -abfss://xxxxx.csv- with the abfs path to your file, which can be found by clicking the 3 dots next to filename

df = spark.read.format("csv").option("header","true").load("abfss://xxxxx.csv")
# df now is a Spark DataFrame containing CSV data from "Files/Alma Analytics/Fund Transactions/Fact - Fund Transactions.csv".
display(df)

StatementMeta(, 8eb58cc2-4822-48df-99c3-ec8247a3fdff, 17, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 36300906-74a9-4ddd-8ba3-30275e245eed)

In [15]:
# this is just example code for some cleanup to the df 
# Code generated by Data Wrangler for PySpark DataFrame

from pyspark.sql import types as T

def clean_data(df):
    # Filter rows based on column: 'PO Line Reference'
    df = df.filter(~df['PO Line Reference'].eqNullSafe("-1"))
    # Change column type to datetime64[ns] for column: 'Transaction Date'
    df = df.withColumn('Transaction Date', df['Transaction Date'].cast(T.TimestampType()))
    # Replace missing values with "<NA>" in columns: 'PO Line Reference', 'Line Price' and 4 other columns
    df = df.fillna(value="<NA>", subset=['PO Line Reference', 'Line Price', 'Transaction Id', 'MMS Id', 'Fund Code', 'Invoice-Number'])
    # Rename column 'Invoice-Number' to 'Invoice Number'
    df = df.withColumnRenamed('Invoice-Number', 'Invoice Number')
    return df

df_clean = clean_data(df)
display(df_clean)

StatementMeta(, 8eb58cc2-4822-48df-99c3-ec8247a3fdff, 18, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 90a474aa-6962-4581-b606-ef9707c4ddb4)

In [16]:
# this writes the clean_df to a delta table in Tables, and overwrites if it already exists
# the printSchema allows you to make sure the columns are all the correct data type

df_clean.write.format("delta").mode('overwrite').option("delta.columnMapping.mode", "name").saveAsTable("dimpol")

df_clean.printSchema()

StatementMeta(, 8eb58cc2-4822-48df-99c3-ec8247a3fdff, 19, Finished, Available, Finished)

root
 |-- PO Line Reference: string (nullable = false)
 |-- Line Price: string (nullable = false)
 |-- Transaction Date: timestamp (nullable = true)
 |-- Transaction Id: string (nullable = false)
 |-- MMS Id: string (nullable = false)
 |-- Fund Code: string (nullable = false)
 |-- Invoice Number: string (nullable = false)

